In [1]:
import pandas as pd
import re
import numpy as np
from clean_data import data_clean

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
def str2float(s):  
    s = str(s).strip().replace(',','')
    s = re.sub('[^0-9]+', '0', s)
    return float(s)

def data_cleaning(raw_dict):
    def str2float(s):
        s = str(s).strip().replace(',','')
        s = re.sub('[^0-9]+', '0', s)
        return float(s)
    
    def remove_sw(words, sw):
        word = [w for w in words if not w in sw]
        return word
    
    def clean_text( col):
        letters_only=(col.apply(lambda x:re.sub(u"\xa0",u" ",x))
        #.apply(lambda x: BeautifulSoup(x).get_text())
        .apply(lambda x:re.sub("[^a-zA-Z]"," ",x))
                 )
        lower_case=letters_only.apply(lambda x: x.lower().split())
        from nltk.corpus import stopwords # Import the stop word list
        stopwords=set(stopwords.words("english"))
        clean_texts = []
        num_texts = col.size
        for i in range( 0, num_texts ):
        # Call our function for each one, and add the result to the list of
        # clean 
            clean_texts.append( " ".join(remove_sw(lower_case[i],stopwords)))
        return clean_texts
    

    
    df=pd.DataFrame(raw_dict)
    df=pd.DataFrame.from_dict(raw_dict,orient='index')
    df.reset_index(inplace=True)
    df.columns=['Petition_Url',"Title","Text","No_Supporters","progression","start"]
    df.start[df.start.str.len() == 0] = 'None\nNone'
    df.start = df.start.str.split('\n')
    df[['start_time', 'start_individual']] =  pd.DataFrame([x for x in df.start])
    df.No_Supporters=df.No_Supporters.apply(str2float)
    df['No_Supporters_log']=np.log10(df.No_Supporters)
    df.Text=clean_text(df.Text)
    df.Title=clean_text(df.Title)
    df['Text_len']=df.Text.str.split(' ').apply(lambda x: len(x))
    df['Title_len']=df.Title.str.split(' ').apply(lambda x: len(x))
    df['Text_str_len']=df.Text.str.split(' ').apply(lambda x: sum(len(w) for w in x)/ len(x))
    df['Title_str_len']=df.Title.str.split(' ').apply(lambda x: sum(len(w) for w in x)/ len(x))
    df['Text_len_p1']=(df['Text_len']<270)*df['Text_len']
    df['Text_len_p2']=(df['Text_len']>270)*df['Text_len']
    return df
    
    
    


In [4]:
raw_dict=pd.read_pickle('vic_pettion_info_dict0-3680_400.pk')

In [5]:
df=pd.DataFrame(raw_dict)
df=pd.DataFrame.from_dict(raw_dict,orient='index')
df.reset_index(inplace=True)
df.columns=['Petition_Url',"Title","Text","No_Supporters","progression","start"]

In [6]:
sample_df=df.loc[0:10,["Title","Text"]]

In [11]:
import pickle
pickle.dump(sample_df, open("sample_df.pickle", 'wb'))

In [7]:
sample_df

,Title,Text
0,Please Sign this Petition! This did not happen...,"March 8, 2011 Scott Osborn age 29, 6’1 and 250..."
1,Halt the Central Coastal California Seismic Im...,The goal of the seismic imaging project is to ...
2,New Jersey Board of Elections: Give me my vote...,I need to exercise my right as an American cit...
3,Allow Mexican American Studies at Santa Maria ...,Mr. Ricardo Valencia is ready to begin teachin...
4,Please continue funding for the South Shore Na...,A precious Long Island resource is under threa...
5,Put my brother George Cox back on Traditional ...,My brother George has chronic liver disease he...
6,Consider removal of the LED signs at 700 Glenw...,"Dear Ms. Taylor,\nThank you for taking the tim..."
7,Throw out the suit filed by Brookridge Country...,There is a law designed to help this gentleman...
8,Space Needle: Please Raise the Pride Flag for ...,"In 2010, the Space Needle made history by rais..."
9,Tell MSNBC to suspend Ed Schultz for calling e...,It's no secret that several of MSNBC's comment...


In [9]:
len(sample_df.Title.str.split(" ")[0])

12

In [12]:
df=data_clean(raw_dict)

In [50]:
df2=df.sort_values(['No_Supporters_log'],ascending=False)

In [51]:
df2 = df2.reset_index(drop=True)

In [33]:
0.4*0.2/0.05/0.6

2.6666666666666674

In [62]:
0.2*3652

730.4000000000001

In [64]:
num1 = np.random.choice([0, 1], size=, p=[.1, .9])
num2 = np.random.choice([0, 1], size=731, p=[.2, .8])


In [53]:
Image=np.append(num1,num2)

In [54]:
len(Image)

3652

In [65]:
num3 = np.random.choice([0, 1], size=1460, p=[.8, .2])
num4 = np.random.choice([0, 1], size=2192, p=[.95, .05])


In [66]:
Tweet=np.append(num3,num4)

In [67]:
df2['Image']=Image
df2['Tweet']=Tweet

In [58]:
df2.columns

Index(['Petition_Url', 'Title', 'Text', 'No_Supporters', 'progression',
       'start', 'start_time', 'start_individual', 'No_Supporters_log',
       'Text_len', 'Title_len', 'Text_str_len', 'Title_str_len', 'Text_len_p1',
       'Text_len_p2', 'Image', 'Tweet'],
      dtype='object')

In [68]:
df2.to_csv('Vic_petition_data.csv')